In [5]:
import os
import yaml

import numpy as np
import pandas as pd

import zarr

from utils.utility_functions import (
    LBP, ORB, FFT, STD, MSE, SSIM, kl_divergence, u_stats, hsd, compare_clusters, plot
)

In [6]:
# Specify cluster pairs
pairs = [(12, 15)]

In [7]:
# Paths and input
out = os.path.join(os.getcwd(), f'output/img_similarity')
if not os.path.exists(out):
    os.makedirs(out)

# Read single-cell sample for VAE analysis
main = pd.read_csv(os.path.join(os.getcwd(), 'input/main.csv'))
vae_clusters = main

# Read image contrast settings for CRC-097 (defined earlier in CyLinter)
if os.path.exists(os.path.join(os.getcwd(), 'input/CRC-097_cylinter_contrast_limits.yml')):
    contrast_limits = yaml.safe_load(
        open(os.path.join(os.getcwd(), 'input/CRC-097_cylinter_contrast_limits.yml'))
    )

# Channel mappings in OME-TIFF
markers = {
    0: 'CD3e', 1: 'CD45RO', 2: 'Keratin', 3: 'aSMA', 4: 'CD4', 5: 'CD45',
    6: 'PD1', 7: 'CD20', 8: 'CD68', 9: 'CD8a', 10: 'CD163', 11: 'FOXP3',
    12: 'PD-L1', 13: 'ECAD', 14: 'Vimentin', 15: 'CDX2', 16: 'LaminABC',
    17: 'Desmin', 18: 'CD31', 19: 'PCNA', 20: 'CollagenIV'
}

# Read combined training, validation, and test thumbnails
root = os.path.join(os.getcwd(), 'input/VAE20/6_latent_space_LD850')
combo_dir = os.path.join(root, 'combined_zarr')
X_combo = zarr.open(combo_dir)

# load data
X_combo = X_combo[:]

print()
for clus_pair in pairs:

    print(clus_pair)

    combo_name = f'{clus_pair[0]}v{clus_pair[1]}'

    save_dir = os.path.join(out, combo_name)
    if not os.path.exists(save_dir):
        os.makedirs(save_dir)

    # Get positional indices of clustering cells (aligned with X_combo)
    clus1 = vae_clusters.index[vae_clusters['VAE20'] == clus_pair[0]]
    clus2 = vae_clusters.index[vae_clusters['VAE20'] == clus_pair[1]]

    # Apply cutoff on median pixel intensity among image channels to determine
    # which markers are expressed by cells in both clusters
    # also assigns X-axis index for plotting
    expressed_channels = {}
    idx = 0
    for ch in range(X_combo.shape[0]):
        imgs1 = X_combo[ch, clus1]
        imgs2 = X_combo[ch, clus2]
        if (np.median(imgs1) > 0) and (np.median(imgs2) > 0):
            expressed_channels[ch] = (markers[ch], idx)
            idx += 1

    # Select number of images from the larger cluster equal to smaller cluster
    # sample_size = min(len(clus1), len(clus2))
    # clus1 = clus1[0:sample_size]
    # clus2 = clus2[0:sample_size]

    clus1 = clus1[0:500]
    clus2 = clus2[0:500]


(12, 15)


In [8]:
expressed_channels = {0: ('CD3e', 0), 5: ('CD45', 1), 9: ('CD8a', 2), 19: ('PCNA', 3)}

for color, metric in zip(['cornflowerblue'], ['MSE']):

    df, cluster_select, sq_err = compare_clusters(
        clus1_name=clus_pair[0], clus1=clus1,
        clus2_name=clus_pair[1], clus2=clus2,
        metric=metric, expressed_channels=expressed_channels,
        markers=markers, X_combo=X_combo, combo_name=combo_name,
        save_dir=save_dir
    )

    # Save dataframe with metric values
    df.reset_index(drop=True, inplace=True)
    df.to_csv(os.path.join(save_dir, f'{metric}.csv'), index=False)

    # Create dataframe of stored t-test stats
    stats = u_stats(
        df=df, metric=metric, cluster_select=cluster_select,
        expressed_channels=expressed_channels, markers=markers,
        combo_name=combo_name
    )

    # Compute one-way ANOVA and Tukey HSD stats
    # hsds = hsd(df=df, metric=metric)

    # Plot metric data
    plot(
        df=df, metric=metric, stats=stats, color=color, sq_err=sq_err,
        combo_name=combo_name, clus_pair=clus_pair, save_dir=save_dir
    )

TypeError: compare_clusters() missing 1 required positional argument: 'window_size'